In [1]:
# Imports for data-preprocessing
import pandas as pd
import numpy as np
from __future__ import print_function

# Import for spliting the data set
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import StandardScaler

# Imports for classificaiton 
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFE
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA 
from sklearn.pipeline import Pipeline
from sklearn import metrics as mt
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler

In [2]:
# Import data
df = pd.read_csv('../data/master.csv')

In [3]:
bins = [17, 34, 49, 64, 90]

group_names = ['17-34', '35-49', '50-64', '65+']

age_groups = pd.cut(df.age, bins, labels=group_names)

df['age_groups'] = age_groups.tolist()

In [4]:
# Delete unwanted variables
del df['workclass']
del df['education']
del df['education_num']
del df['marital_status']
del df['occupation']
del df['native_country'] 
del df['age']

In [7]:
# Split the data into traning (80%) and test set (20%)
# We are using stratified cross validation here because the majority of the
#    individuals in the variable race are white

if 'age_groups' in df:
    y = df['age_groups'] #get values we need 
    del df['age_groups']        #get rid of the class label
    X = df.values                  #use everything else to predict 
    
X = pd.get_dummies(df).values

scl = StandardScaler()
# X = scl.fit_transform(X)

# Split the data into 20% Test and 80% Train
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.20, random_state=111)
sss.get_n_splits(X, y) #retreving the splits

10

In [43]:
# y = y.fillna("")

# for train_index, test_index in sss.split(X,y.tolist()):
#     print("TRAIN:", train_index, "TEST:", test_index)
#     X_train, X_test = X[train_index], X[test_index]
#     y_train, y_test = y[train_index], y[test_index]

TRAIN: [ 8492   773 13118 ..., 35980 33511 35588] TEST: [19692 27047 14102 ...,  5491 27134 25133]
TRAIN: [35754 44603 33516 ...,  9966 39949 27930] TEST: [ 6576 30339 39221 ..., 26307 20282 14216]
TRAIN: [ 9692 36813  2848 ..., 16610 48026 24110] TEST: [17088 44904 33742 ..., 39410 34564 24713]
TRAIN: [18717 17405 28960 ..., 17899 20181 28803] TEST: [29770  5237 39991 ..., 19858 25079 39382]
TRAIN: [31784 43606 33743 ..., 43070 11410 40058] TEST: [ 5991  5420 37845 ..., 23131 38821 38455]
TRAIN: [10324 22662 46232 ..., 13002  5850 47428] TEST: [35217 41009 45992 ..., 19937 37685 43961]
TRAIN: [10245 34540 19934 ...,  4642 48455  1175] TEST: [46966 10993 27579 ..., 31110 48759 43327]
TRAIN: [39885  4332  3592 ..., 44157 11660 46607] TEST: [31259 23034  5609 ..., 20253 24091 12492]
TRAIN: [17705 32836 34332 ..., 30752 31903 12175] TEST: [24823 12082 42233 ..., 14850 38281 46484]
TRAIN: [ 9054 18693 13929 ...,  8118 28501 10876] TEST: [37717 40166 28516 ...,  4861 47589  9591]


In [8]:
y = y.fillna("")
model = LogisticRegression(penalty='l2', C=1.0)
rfe = RFE(model, 10)
fit = rfe.fit(X, y)


TypeError: '>' not supported between instances of 'float' and 'str'

In [51]:
# print("Num Features:", fit.n_features_)
# features = pd.Series(fit.support_, index=X.columns)
# print("Selected Features: \n", features[features==True])
# print("Feature Ranking:", fit.ranking_)

Num Features: 10


AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [9]:
iter_num=0

# the indices are the rows used for training and testing in each iteration
for train_indices, test_indices in sss.split(X,y.tolist()): 
    X_train = X[train_indices]  # train indices for X
    y_train = y[train_indices]  # train indices for y
    
    X_test = X[test_indices]    # test indices for X
    y_test = y[test_indices]    # test indices for y
    
    # train the reusable logisitc regression model on the training data
    rfe.fit(X_train,y_train)  # train object
    y_hat_rfe = rfe.predict(X_test) # get test set precitions

    # accuracy and confusion matrix for this iterations of training/testing
    acc_rfe1 = mt.accuracy_score(y_test,y_hat_rfe)
    conf_rfe1 = mt.confusion_matrix(y_test,y_hat_rfe)
    print("====Iteration",iter_num," ====")
    print("accuracy", acc_rfe1 )
    print("confusion matrix\n",conf_rfe1)
    iter_num+=1


====Iteration 0  ====
accuracy 0.573241887604
confusion matrix
 [[   0  117    1    0    1]
 [   0 2666 1404    9    4]
 [   0  617 2744   53   18]
 [   0  175 1373  128   42]
 [   0   67  231   57   62]]
====Iteration 1  ====
accuracy 0.571911147507
confusion matrix
 [[   0  119    0    0    0]
 [   0 2647 1425   10    1]
 [   0  619 2749   54   10]
 [   0  185 1365  147   21]
 [   0   52  230   91   44]]
====Iteration 2  ====
accuracy 0.563824342307
confusion matrix
 [[   0  118    0    1    0]
 [   0 2607 1364  112    0]
 [   0  616 2650  164    2]
 [   0  142 1329  243    4]
 [   0   32  227  150    8]]
====Iteration 3  ====
accuracy 0.565155082404
confusion matrix
 [[   0  119    0    0    0]
 [   0 2631 1436   11    5]
 [   0  664 2707   52    9]
 [   0  178 1375  138   27]
 [   0   60  206  106   45]]
====Iteration 4  ====
accuracy 0.572218241376
confusion matrix
 [[   0  119    0    0    0]
 [   0 2682 1385   10    6]
 [   0  656 2707   49   20]
 [   0  181 1353  138   46]
 [  

In [10]:
# Get the mean accuracy

# Load the accuracies
accuracies_rfe = cross_val_score(rfe, X, y=y, cv=sss) # this also can help with parallelism

# Print out the mean 
mean_rfe = np.mean(accuracies_rfe)
print("The mean accuracy for this model is ", mean_rfe)

The mean accuracy for this model is  0.566751970519


In [60]:
model2 = LogisticRegression(penalty='l2', C=0.5)
rfe2 = RFE(model2, 10)
fit2 = rfe.fit(X, y)
# print("Num Features:", fit2.n_features_)
# features2 = pd.Series(fit2.support_, index = X.columns)
# print("Selected Features: \n", features2[features2==True])
# print("Selected Features:", fit2.support_)
# print("Feature Ranking:", fit2.ranking_)

In [61]:
iter_num=0
# the indices are the rows used for training and testing in each iteration
for train_indices, test_indices in sss.split(X,y): 
    X_train = X[train_indices]  # train indices for X
    y_train = y[train_indices]  # train indices for y
    
    X_test = X[test_indices]    # test indices for X
    y_test = y[test_indices]    # test indices for y
    
    # train the reusable logisitc regression model on the training data
    rfe2.fit(X_train,y_train)  # train object
    y_hat_rfe2 = rfe2.predict(X_test) # get test set precitions

    # accuracy and confusion matrix for this iterations of training/testing
    acc_rfe2 = mt.accuracy_score(y_test,y_hat_rfe2)
    conf_rfe2 = mt.confusion_matrix(y_test,y_hat_rfe2)
    print("====Iteration",iter_num," ====")
    print("accuracy", acc_rfe2 )
    print("confusion matrix\n",conf_rfe2)
    iter_num+=1


====Iteration 0  ====
accuracy 0.570682772034
confusion matrix
 [[   0  117    1    0    1]
 [   0 2651 1420   10    2]
 [   0  612 2753   57   10]
 [   0  181 1381  129   27]
 [   0   67  237   71   42]]
====Iteration 1  ====
accuracy 0.57088750128
confusion matrix
 [[   0  119    0    0    0]
 [   0 2641 1432    9    1]
 [   0  608 2764   53    7]
 [   0  189 1377  132   20]
 [   0   64  232   81   40]]
====Iteration 2  ====
accuracy 0.57017094892
confusion matrix
 [[   0  118    1    0    0]
 [   0 2626 1423   32    2]
 [   0  628 2737   60    7]
 [   0  152 1385  160   21]
 [   0   33  202  135   47]]
====Iteration 3  ====
accuracy 0.563619613062
confusion matrix
 [[   0  119    0    0    0]
 [   0 2622 1445   14    2]
 [   0  661 2715   51    5]
 [   0  184 1382  129   23]
 [   0   72  207   98   40]]
====Iteration 4  ====
accuracy 0.571808782885
confusion matrix
 [[   0  119    0    0    0]
 [   0 2676 1391   11    5]
 [   0  646 2720   56   10]
 [   0  193 1361  139   25]
 [   0

In [62]:

# Load the accuracies
accuracies_rfe2 = cross_val_score(rfe2, X, y=y, cv=sss) # this also can help with parallelism

# Print out the mean 
mean_rfe2 = np.mean(accuracies_rfe2)
print("The mean accuracy for this model is ", mean_rfe2)

The mean accuracy for this model is  0.566629132972


In [12]:
model3 = LogisticRegression(penalty='l2', C=0.5)
rfe3 = RFE(model3, 20)
fit3 = rfe.fit(X, y)

In [13]:
iter_num=0
# the indices are the rows used for training and testing in each iteration
for train_indices, test_indices in sss.split(X,y): 
    X_train = X[train_indices]  # train indices for X
    y_train = y[train_indices]  # train indices for y
    
    X_test = X[test_indices]    # test indices for X
    y_test = y[test_indices]    # test indices for y
    
    # train the reusable logisitc regression model on the training data
    rfe3.fit(X_train,y_train)  # train object
    y_hat_rfe3 = rfe3.predict(X_test) # get test set precitions

    # accuracy and confusion matrix for this iterations of training/testing
    acc_rfe3 = mt.accuracy_score(y_test,y_hat_rfe3)
    conf_rfe3 = mt.confusion_matrix(y_test,y_hat_rfe3)
    print("====Iteration",iter_num," ====")
    print("accuracy", acc_rfe3 )
    print("confusion matrix\n",conf_rfe3)
    iter_num+=1


====Iteration 0  ====
accuracy 0.575391544682
confusion matrix
 [[   0  117    0    1    1]
 [   0 2850 1137   91    5]
 [   0  784 2451  182   15]
 [   0  218 1194  249   57]
 [   0   59  177  110   71]]
====Iteration 1  ====
accuracy 0.573344252226
confusion matrix
 [[   0  119    0    0    0]
 [   0 2736 1242  104    1]
 [   0  677 2544  205    6]
 [   0  220 1200  276   22]
 [   0   52  185  135   45]]
====Iteration 2  ====
accuracy 0.57385607534
confusion matrix
 [[   0  119    0    0    0]
 [   0 2729 1243  108    3]
 [   0  702 2568  155    7]
 [   0  218 1220  255   25]
 [   0   70  165  128   54]]
====Iteration 3  ====
accuracy 0.566690551745
confusion matrix
 [[   0  119    0    0    0]
 [   0 2726 1256   99    2]
 [   0  733 2496  195    8]
 [   0  218 1196  270   34]
 [   0   64  166  143   44]]
====Iteration 4  ====
accuracy 0.574879721568
confusion matrix
 [[   0  119    0    0    0]
 [   0 2854 1194   30    5]
 [   0  802 2518  100   12]
 [   0  240 1258  187   33]
 [   

In [14]:
# Load the accuracies
accuracies_rfe3 = cross_val_score(rfe3, X, y=y, cv=sss) # this also can help with parallelism

# Print out the mean 
mean_rfe3 = np.mean(accuracies_rfe3)
print("The mean accuracy for this model is ", mean_rfe3)

The mean accuracy for this model is  0.571235540997


In [15]:
model4 = LogisticRegression(penalty='l2', C=1.0)
rfe4 = RFE(model4, 30)
fit4 = rfe.fit(X, y)

In [16]:
iter_num=0
# the indices are the rows used for training and testing in each iteration
for train_indices, test_indices in sss.split(X,y): 
    X_train = X[train_indices]  # train indices for X
    y_train = y[train_indices]  # train indices for y
    
    X_test = X[test_indices]    # test indices for X
    y_test = y[test_indices]    # test indices for y
    
    # train the reusable logisitc regression model on the training data
    rfe4.fit(X_train,y_train)  # train object
    y_hat_rfe4 = rfe4.predict(X_test) # get test set precitions

    # accuracy and confusion matrix for this iterations of training/testing
    acc_rfe4 = mt.accuracy_score(y_test,y_hat_rfe4)
    conf_rfe4 = mt.confusion_matrix(y_test,y_hat_rfe4)
    print("====Iteration",iter_num," ====")
    print("accuracy", acc_rfe4 )
    print("confusion matrix\n",conf_rfe4)
    iter_num+=1

====Iteration 0  ====
accuracy 0.574777356945
confusion matrix
 [[   0  117    1    0    1]
 [   0 2847 1199   34    3]
 [   0  781 2534  106   11]
 [   0  238 1258  186   36]
 [   0   78  195   96   48]]
====Iteration 1  ====
accuracy 0.579281400348
confusion matrix
 [[   0  119    0    0    0]
 [   0 2855 1192   34    2]
 [   0  755 2565  104    8]
 [   0  245 1258  189   26]
 [   0   56  202  109   50]]
====Iteration 2  ====
accuracy 0.574163169209
confusion matrix
 [[   0  119    0    0    0]
 [   0 2828 1216   35    4]
 [   0  803 2554   68    7]
 [   0  264 1253  170   31]
 [   0   73  181  106   57]]
====Iteration 3  ====
accuracy 0.571501689016
confusion matrix
 [[   0  119    0    0    0]
 [   0 2824 1158   99    2]
 [   0  785 2454  184    9]
 [   0  241 1188  253   36]
 [   0   62  167  136   52]]
====Iteration 4  ====
accuracy 0.576005732419
confusion matrix
 [[   0  119    0    0    0]
 [   0 2865 1177   36    5]
 [   0  808 2504  105   15]
 [   0  237 1243  196   42]
 [  

In [17]:
# Load the accuracies
accuracies_rfe4 = cross_val_score(rfe4, X, y=y, cv=sss) # this also can help with parallelism

# Print out the mean 
mean_rfe4 = np.mean(accuracies_rfe4)
print("The mean accuracy for this model is ", mean_rfe4)

The mean accuracy for this model is  0.572566281093


In [18]:
# LR model #1
lr_clf = LogisticRegression(penalty='l2', C=1.0) # get object

iter_num=0
# the indices are the rows used for training and testing in each iteration
for train_indices, test_indices in sss.split(X,y): 
    X_train = X[train_indices]  # train indices for X
    y_train = y[train_indices]  # train indices for y
    
    X_test = X[test_indices]    # test indices for X
    y_test = y[test_indices]    # test indices for y
    
    # train the reusable logisitc regression model on the training data
    lr_clf.fit(X_train,y_train)  # train object
    y_hat = lr_clf.predict(X_test) # get test set precitions

    # accuracy and confusion matrix for this iterations of training/testing
    acc = mt.accuracy_score(y_test,y_hat)
    conf = mt.confusion_matrix(y_test,y_hat)
    print("====Iteration",iter_num," ====")
    print("accuracy", acc )
    print("confusion matrix\n",conf)
    iter_num+=1

====Iteration 0  ====
accuracy 0.568942573447
confusion matrix
 [[   0  118    1    0    0]
 [   0 2735 1320   18   10]
 [   0  702 2664   50   16]
 [   0  272 1322   79   45]
 [   0  117  164   56   80]]
====Iteration 1  ====
accuracy 0.572115876753
confusion matrix
 [[   2  117    0    0    0]
 [   1 2779 1272   20   11]
 [   0  757 2602   43   30]
 [   0  290 1291   84   53]
 [   0   87  163   45  122]]
====Iteration 2  ====
accuracy 0.570989865902
confusion matrix
 [[   7  112    0    0    0]
 [   7 2802 1248   13   13]
 [   0  788 2618   15   11]
 [   0  271 1359   26   62]
 [   0   77  180   35  125]]
====Iteration 3  ====
accuracy 0.570068584297
confusion matrix
 [[   0  119    0    0    0]
 [   1 2969 1080   25    8]
 [   0  950 2405   60   17]
 [   0  316 1261   87   54]
 [   0   65  160   84  108]]
====Iteration 4  ====
accuracy 0.575801003173
confusion matrix
 [[   5  114    0    0    0]
 [   3 2872 1175   24    9]
 [   0  827 2508   71   26]
 [   0  247 1298  101   72]
 [  

In [19]:
# Load the accuracies
accuracies_lr_clf = cross_val_score(lr_clf, X, y=y, cv=sss) # this also can help with parallelism

# Print out the mean 
mean_lr_clf = np.mean(accuracies_lr_clf)
print("The mean accuracy for this model is ", mean_lr_clf)

The mean accuracy for this model is  0.570017401986
